In [1]:
import ibm_db

In [2]:
#Replace the placeholder values with your actual Db2 hostname, username, and password:
dsn_hostname = "dashdb-txn-sbox-yp-lon02-02.services.eu-gb.bluemix.net" # e.g.: "dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net"
dsn_uid = "pdb33904"        # e.g. "abc12345"
dsn_pwd = "3zczs5dch^67p6w5"      # e.g. "7dBZ3wWt9XN6$o0J"

dsn_driver = "{IBM DB2 ODBC DRIVER}"
dsn_database = "BLUDB"            # e.g. "BLUDB"
dsn_port = "50000"                # e.g. "50000" 
dsn_protocol = "TCPIP"            # i.e. "TCPIP"

In [3]:
#DO NOT MODIFY THIS CELL. Just RUN it with Shift + Enter
#Create the dsn connection string
dsn = (
    "DRIVER={0};"
    "DATABASE={1};"
    "HOSTNAME={2};"
    "PORT={3};"
    "PROTOCOL={4};"
    "UID={5};"
    "PWD={6};").format(dsn_driver, dsn_database, dsn_hostname, dsn_port, dsn_protocol, dsn_uid, dsn_pwd)

#print the connection string to check correct values are specified
print(dsn)

DRIVER={IBM DB2 ODBC DRIVER};DATABASE=BLUDB;HOSTNAME=dashdb-txn-sbox-yp-lon02-02.services.eu-gb.bluemix.net;PORT=50000;PROTOCOL=TCPIP;UID=pdb33904;PWD=3zczs5dch^67p6w5;


In [4]:
#DO NOT MODIFY THIS CELL. Just RUN it with Shift + Enter
#Create database connection

try:
    conn = ibm_db.connect(dsn, "", "")
    print ("Connected to database: ", dsn_database, "as user: ", dsn_uid, "on host: ", dsn_hostname)

except:
    print ("Unable to connect: ", ibm_db.conn_errormsg() )

Connected to database:  BLUDB as user:  pdb33904 on host:  dashdb-txn-sbox-yp-lon02-02.services.eu-gb.bluemix.net


In [5]:
#Retrieve Metadata for the Database Server
server = ibm_db.server_info(conn)

print ("DBMS_NAME: ", server.DBMS_NAME)
print ("DBMS_VER:  ", server.DBMS_VER)
print ("DB_NAME:   ", server.DB_NAME)

DBMS_NAME:  DB2/LINUXX8664
DBMS_VER:   11.01.0404
DB_NAME:    BLUDB


In [6]:
#Retrieve Metadata for the Database Client / Driver
client = ibm_db.client_info(conn)

print ("DRIVER_NAME:          ", client.DRIVER_NAME) 
print ("DRIVER_VER:           ", client.DRIVER_VER)
print ("DATA_SOURCE_NAME:     ", client.DATA_SOURCE_NAME)
print ("DRIVER_ODBC_VER:      ", client.DRIVER_ODBC_VER)
print ("ODBC_VER:             ", client.ODBC_VER)
print ("ODBC_SQL_CONFORMANCE: ", client.ODBC_SQL_CONFORMANCE)
print ("APPL_CODEPAGE:        ", client.APPL_CODEPAGE)
print ("CONN_CODEPAGE:        ", client.CONN_CODEPAGE)

DRIVER_NAME:           libdb2.a
DRIVER_VER:            11.05.0400
DATA_SOURCE_NAME:      BLUDB
DRIVER_ODBC_VER:       03.51
ODBC_VER:              03.01.0000
ODBC_SQL_CONFORMANCE:  EXTENDED
APPL_CODEPAGE:         1208
CONN_CODEPAGE:         1208


In [7]:
import pandas
import ibm_db_dbi
#connection for pandas
pconn = ibm_db_dbi.Connection(conn)

#### Find the total number of crimes recorded in the CRIME table.

In [8]:
query1 = 'select COUNT(*) from "CRIME_DATA";'
df1 = pandas.read_sql(query1, pconn)
df1

,1
0,533.0


#### List community areas with per capita income less than 11000.

In [9]:
query2 = 'select "COMMUNITY_AREA_NAME" from "CENSUS_DATA" where "PER_CAPITA_INCOME" < 11000;'
df2 = pandas.read_sql(query2, pconn)
df2

,COMMUNITY_AREA_NAME
0,West Garfield Park
1,South Lawndale
2,Fuller Park
3,Riverdale


#### List all case numbers for crimes involving minors?

In [14]:
query3 = 'select "CASE_NUMBER" from "CRIME_DATA" where "DESCRIPTION" like \'%MINOR%\';'
df3 = pandas.read_sql(query3, pconn)
df3

,CASE_NUMBER
0,HL266884
1,HK238408


#### List all kidnapping crimes involving a child?(children are not considered minors for the purposes of crime analysis)

In [26]:
query4 = 'select "ID", "CASE_NUMBER", "DATE", "BLOCK", "IUCR" from "CRIME_DATA" where "PRIMARY_TYPE" like \'%KIDNAPPING%\';'
df4 = pandas.read_sql(query4, pconn)
df4

,ID,CASE_NUMBER,DATE,BLOCK,IUCR
0,5276766,HN144152,2007-01-26,050XX W VAN BUREN ST,1792


#### What kind of crimes were recorded at schools?

In [30]:
query5 = 'select DISTINCT(PRIMARY_TYPE) AS CRIMES from "CRIME_DATA" where "LOCATION_DESCRIPTION" like \'%SCHOOL%\';'
df5 = pandas.read_sql(query5, pconn)
df5

,CRIMES
0,ASSAULT
1,BATTERY
2,CRIMINAL DAMAGE
3,CRIMINAL TRESPASS
4,NARCOTICS
5,PUBLIC PEACE VIOLATION


#### List the average safety score for all types of schools.

In [35]:
query6 = 'select AVG(SAFETY_SCORE) from "PUBLIC_SCHOOLS";'
df6 = pandas.read_sql(query6, pconn)
df6

,1
0,49.504873


#### List 5 community areas with highest % of households below poverty line.

In [38]:
query7 = 'select "COMMUNITY_AREA_NAME" from "CENSUS_DATA" ORDER BY "PERCENT_HOUSEHOLDS_BELOW_POVERTY" DESC LIMIT 5;'
df7 = pandas.read_sql(query7, pconn)
df7

,COMMUNITY_AREA_NAME
0,Riverdale
1,Fuller Park
2,Englewood
3,North Lawndale
4,East Garfield Park


#### Which community area(number) is most crime prone?

In [45]:
query8 = 'select "COMMUNITY_AREA_NUMBER" from "CRIME_DATA" GROUP BY "COMMUNITY_AREA_NUMBER" ORDER BY COUNT(*) DESC LIMIT 1;'
df8 = pandas.read_sql(query8, pconn)
df8

,COMMUNITY_AREA_NUMBER
0,25


#### Use a sub-query to find the name of the community area with highest hardship index.

In [50]:
query9 = 'select "COMMUNITY_AREA_NAME" from "CENSUS_DATA" where "HARDSHIP_INDEX" = (select MAX(HARDSHIP_INDEX) from "CENSUS_DATA");'
df9 = pandas.read_sql(query9, pconn)
df9

,COMMUNITY_AREA_NAME
0,Riverdale


#### Use a sub-query to determine the Community Area Name with most number of crimes?

In [75]:
query10 = 'SELECT COMMUNITY_AREA_NAME, COMMUNITY_AREA_NUMBER FROM CENSUS_DATA WHERE COMMUNITY_AREA_NUMBER IN \
(SELECT COMMUNITY_AREA_NUMBER FROM CRIME_DATA GROUP BY COMMUNITY_AREA_NUMBER ORDER BY COUNT(COMMUNITY_AREA_NUMBER) DESC LIMIT 1);'
df10 = pandas.read_sql(query10, pconn)
df10

,COMMUNITY_AREA_NAME,COMMUNITY_AREA_NUMBER
0,Austin,25
